# 인덱싱·필터링·정제 실전(조회/조건/정렬/클리닝)

# 1) 인덱싱
데이터프레임에서 특정 행(row)과 열(column)을 선택하는 방법

분석의 정확도를 결정하는 기본중 기본이기 때문에 제대로 하지 않으면 설령 통계를 제대로 하더라도 결과가 흔들릴수도 있다.

## 1) 필요한 행만 가져오기(Row 선택)
실제 데이터는 대부분 “전체”를 분석하지 않고 보고자 하는, 얻고자 하는 데이터만을 대상으로 한다.
예를 들어
- 특정 날짜(어제/이번 주)만
- 특정 지점만
- 결제 완료 건만 등
이때 행 선택이 잘못되면(결제 실패 섞임, 기간 섞임 등) 매출/통계가 통째로 왜곡될 수 있다.


## 2)  필요한 열만 남기기(Column 선택)
분석에 필요한 핵심 컬럼만 남겨 불필요한 컬럼을 지운다
- 작업이 단순해짐
- 실수(컬럼 혼동)가 줄어듬
- 정제/집계/시각화가 더 빨라짐 (데이터 자체가 축소되기 떄문)

## 3) 조건에 맞는 데이터만 보기(Filter)
분석은 보통 비교(지점별/시간대별/메뉴별) 가 핵심, 비교를 하려면 먼저 조건으로 데이터를 나눠야한다.
- A점 vs B점
- 오전 vs 오후
- 평일 vs 주말
이 조건을 정확히 구해야 비교 분석이 가능해진다

# 2)  .loc vs .iloc

## .loc = 라벨(이름) 기반 (행/열 이름으로 접근)
이 이름을 가진 행/열을 가져와라
- 행/열 이름을 사용
- 문자열 컬럼명 사용 가능
- 슬라이싱 시 끝 값 포함
- 조건 필터링과 함께 자주 사용됨


## .iloc = 위치(순서) 기반 (0번째, 1번째…로 접근)
몇 번째에 있는 행/열을 가져와라
- 정수 인덱스만 사용
- 파이썬 리스트 인덱싱과 동일한 규칙
- 슬라이싱 시 끝 값 미포함
- 구조가 확실할 때 사용하기 좋음    

In [65]:
import pandas as pd

In [66]:
df = pd.DataFrame(
    {"menu": ["Latte", "Americano", "Mocha"], "price": [5000, 4500, 5500]},
    index=["A001", "A002", "A003"]
)
df

,menu,price
A001,Latte,5000
A002,Americano,4500
A003,Mocha,5500


In [67]:
df.loc["A002"]  # <- 라벨로 찾기

menu     Americano
price         4500
Name: A002, dtype: object

In [68]:
df.iloc[1]      # <- 위치로 찾기

menu     Americano
price         4500
Name: A002, dtype: object

## 언제 쓰는게 자연스러운가?

In [69]:
# ------------------------------------------------------------
# 1) 샘플 데이터 만들기 (카페 매출)
#    - date: 날짜(나중에 인덱스로도 써볼 예정)
#    - paid: 결제 완료 여부(True/False)
# ------------------------------------------------------------
df = pd.DataFrame({
    "date":  ["2026-01-01","2026-01-02","2026-01-03","2026-01-04","2026-01-05","2026-01-06","2026-01-07"],
    "store": ["A","A","B","A","B","A","B"],
    "menu":  ["Latte","Americano","Mocha","Latte","Americano","Mocha","Latte"],
    "qty":   [1,2,1,3,1,2,2],
    "paid":  [True, True, False, True, True, False, True]
})
df

,date,store,menu,qty,paid
0,2026-01-01,A,Latte,1,True
1,2026-01-02,A,Americano,2,True
2,2026-01-03,B,Mocha,1,False
3,2026-01-04,A,Latte,3,True
4,2026-01-05,B,Americano,1,True
5,2026-01-06,A,Mocha,2,False
6,2026-01-07,B,Latte,2,True


### .loc 이 자연스러운 상황
- .loc이 자연스러운 상황
    - 조건 필터링과 같이 쓸 때 (가장 흔함)
        - "결제 완료(True)인 행만 + 필요한 컬럼만"
    - 인덱스/컬럼 이름이 의미가 있을 때
        - 예: 날짜가 인덱스인 경우("2026-01-01" 같은 라벨)
        - 예: df.loc["2026-01-01":"2026-01-07"]처럼 라벨 범위로 자를 때

In [70]:
# ------------------------------------------------------------
# 2) 조건 필터링 + 필요한 컬럼만
#    - "결제 완료(True)인 행만" 가져오고
#    - 그 중에서 date/store/menu/qty 컬럼만 보고 싶을 때
#    - 패턴: df.loc[조건, [컬럼들]]
# ------------------------------------------------------------
loc_filtered = df.loc[df["paid"] == True, ["date", "store", "menu", "qty"]]
loc_filtered

,date,store,menu,qty
0,2026-01-01,A,Latte,1
1,2026-01-02,A,Americano,2
3,2026-01-04,A,Latte,3
4,2026-01-05,B,Americano,1
6,2026-01-07,B,Latte,2


In [71]:
# ------------------------------------------------------------
# 3) 인덱스(라벨)가 의미 있을 때
#    - date를 인덱스로 바꾸면, 날짜 자체가 '행 이름표(라벨)'가 된다.
# ------------------------------------------------------------
df_dt = df.set_index("date")
df_dt

,store,menu,qty,paid
date,,,,
2026-01-01,A,Latte,1,True
2026-01-02,A,Americano,2,True
2026-01-03,B,Mocha,1,False
2026-01-04,A,Latte,3,True
2026-01-05,B,Americano,1,True
2026-01-06,A,Mocha,2,False
2026-01-07,B,Latte,2,True


In [72]:
# ------------------------------------------------------------
# 4) 라벨(날짜) 범위로 자르기
#    - "2026-01-01 ~ 2026-01-04" 구간만 보고 싶을 때
#    - 주의: loc은 라벨 기준이라, 범위 슬라이싱에서 끝 라벨이 포함되는 경우가 많다.
# ------------------------------------------------------------
loc_range = df_dt.loc["2026-01-01":"2026-01-04", ["store", "menu", "qty", "paid"]]
loc_range

,store,menu,qty,paid
date,,,,
2026-01-01,A,Latte,1,True
2026-01-02,A,Americano,2,True
2026-01-03,B,Mocha,1,False
2026-01-04,A,Latte,3,True


### .iloc 이 자연스러운 상황
- .iloc이 자연스러운 상황
    - 위에서 몇 줄만 보기
        - 예: 첫 5행, 특정 구간(0~9행)
    - 위치 기반으로 정확히 잘라야 할 때
        - 예: df.iloc[0:10] (처음 10개)
    - 컬럼 이름이 헷갈릴 때 "몇 번째 열로 빠르게 확인할 때
        - 예: df.iloc[:, 0] (첫 번째 열)

In [73]:
# ------------------------------------------------------------
# 5) 위에서 몇 줄만 보기 (첫 5행)
#    - iloc은 '순서(위치)' 기준: 0번째~4번째(총 5행)
#    - 주의: iloc 슬라이싱은 파이썬 규칙대로 끝(5)이 미포함
# ------------------------------------------------------------
iloc_head5 = df.iloc[0:5]
iloc_head5

,date,store,menu,qty,paid
0,2026-01-01,A,Latte,1,True
1,2026-01-02,A,Americano,2,True
2,2026-01-03,B,Mocha,1,False
3,2026-01-04,A,Latte,3,True
4,2026-01-05,B,Americano,1,True


In [74]:
# ------------------------------------------------------------
# 6) 위치로 정확히 자르기
#    - 처음 3개 행만(0,1,2)
# ------------------------------------------------------------
iloc_first3 = df.iloc[0:3]
iloc_first3


,date,store,menu,qty,paid
0,2026-01-01,A,Latte,1,True
1,2026-01-02,A,Americano,2,True
2,2026-01-03,B,Mocha,1,False


In [75]:
# ------------------------------------------------------------
# 7) 컬럼 이름이 헷갈릴 때 "몇 번째 열"로 빠르게 확인
#    - df.iloc[:, 0]  => 모든 행(:,)에서 첫 번째 열(0번째 열)만
# ------------------------------------------------------------
iloc_first_col = df.iloc[:, 0]
iloc_first_col

0    2026-01-01
1    2026-01-02
2    2026-01-03
3    2026-01-04
4    2026-01-05
5    2026-01-06
6    2026-01-07
Name: date, dtype: object

In [76]:
# ------------------------------------------------------------
# 8) 노트북에서 print 없이 여러 결과를 한 번에 보기
#    - 셀의 마지막 줄에 (원하는 것들) 을 리스트/튜플로 묶어두면 자동 출력된다.
# ------------------------------------------------------------
df, loc_filtered, df_dt, loc_range, iloc_head5, iloc_first3, iloc_first_col

(         date store       menu  qty   paid
 0  2026-01-01     A      Latte    1   True
 1  2026-01-02     A  Americano    2   True
 2  2026-01-03     B      Mocha    1  False
 3  2026-01-04     A      Latte    3   True
 4  2026-01-05     B  Americano    1   True
 5  2026-01-06     A      Mocha    2  False
 6  2026-01-07     B      Latte    2   True,
          date store       menu  qty
 0  2026-01-01     A      Latte    1
 1  2026-01-02     A  Americano    2
 3  2026-01-04     A      Latte    3
 4  2026-01-05     B  Americano    1
 6  2026-01-07     B      Latte    2,
            store       menu  qty   paid
 date                                   
 2026-01-01     A      Latte    1   True
 2026-01-02     A  Americano    2   True
 2026-01-03     B      Mocha    1  False
 2026-01-04     A      Latte    3   True
 2026-01-05     B  Americano    1   True
 2026-01-06     A      Mocha    2  False
 2026-01-07     B      Latte    2   True,
            store       menu  qty   paid
 date        

## 슬라이싱 실수 포인트(중요): 끝 포함 여부
- .loc 슬라이싱은 "끝을 포함"하는 경우가 많다
- .iloc 슬라이싱은 파이썬 규칙대로 "끝 미포함"이다
 
### 초보자용 예시(개념 이해용)
인덱스가 0,1,2,3,4일 때

- df.loc[0:2] → 0,1,2 포함 (대개 3행)
- df.iloc[0:2] → 0,1 만(2행)

In [ ]:
df = pd.DataFrame(
    {"menu": ["Latte", "Americano", "Mocha", "Tea", "Cake"],
    "price": [5000, 4500, 5500, 4000, 6000]},
    index=[0, 1, 2, 3, 4]  # 인덱스(라벨)가 0~4인 상태
)
df

,menu,price
0,Latte,5000
1,Americano,4500
2,Mocha,5500
3,Tea,4000
4,Cake,6000


In [78]:
#  .loc 슬라이싱: 끝(3)을 포함할 수 있음 -> 0,1,2,3 (4개)
loc_slice = df.loc[0:3]
loc_slice

,menu,price
0,Latte,5000
1,Americano,4500
2,Mocha,5500
3,Tea,4000


In [79]:
#  .iloc 슬라이싱: 끝(3) 미포함 -> 0,1,2 (3개)
iloc_slice = df.iloc[0:3]
iloc_slice

,menu,price
0,Latte,5000
1,Americano,4500
2,Mocha,5500


### "마지막 포함/미포함" 때문에 생기는 대표 실수

- “처음 10개만 가져오자” 할 때
    - iloc[0:10]은 정확히 10개
    - loc[0:10]은 11개가 될 수 있음

In [80]:
# ------------------------------------------------------------
# 초보자용 예시: .loc vs .iloc 슬라이싱(끝 포함/미포함)
# 인덱스가 0,1,2,3,4일 때
# - df.loc[0:2]  : 0,1,2 포함 -> 3행
# - df.iloc[0:2] : 0,1만     -> 2행 (끝 2는 미포함)
# ------------------------------------------------------------

df = pd.DataFrame(
    {"menu": ["Latte", "Americano", "Mocha", "Tea", "Cake"]},
    index=[0, 1, 2, 3, 4]
)
df

,menu
0,Latte
1,Americano
2,Mocha
3,Tea
4,Cake


In [81]:
loc_0_2  = df.loc[0:2]   # 0,1,2 (3행)
loc_0_2

,menu
0,Latte
1,Americano
2,Mocha


In [82]:
iloc_0_2 = df.iloc[0:2]  # 0,1   (2행)
iloc_0_2

,menu
0,Latte
1,Americano


In [83]:
# ------------------------------------------------------------
# 대표 실수: "처음 10개만 가져오자"
# - iloc[0:10]은 파이썬 슬라이스 규칙(끝 미포함)이라 정확히 10개
# - loc[0:10]은 라벨 구간이라 0~10을 포함할 수 있어 11개가 될 수 있음
# ------------------------------------------------------------

df_big = pd.DataFrame({"x": range(100)}, index=range(100))

first10_iloc = df_big.iloc[0:10]  # 0~9 -> 10개
first10_loc  = df_big.loc[0:10]   # 0~10 -> 11개(가능)

print(('loc:'),len(first10_loc))
print(('iloc:'),len(first10_iloc))

loc: 11
iloc: 10


In [84]:
first10_loc.head()

,x
0,0
1,1
2,2
3,3
4,4


In [85]:
first10_iloc.head()

,x
0,0
1,1
2,2
3,3
4,4


### 기억하기 쉬운 암기 팁

- loc = label(이름표)
- iloc = integer location(정수 위치)
- 슬라이싱은 이렇게 외우기:
    - loc는 라벨 구간이라 끝 포함 가능성 높음
    - iloc는 파이썬 슬라이스 그대로(끝 미포함)

# 3) 행/열 선택 패턴(실무에서 자주 사용됨)

pandas에서 "어떤 열을 선택했는지"에 따라 결과가 Series(1차원) 로 나오기도 하고, DataFrame(2차원) 으로 나오기도 한다.

문제는 이 둘이 지원하는 메소드/속성/출력 형태가 다르고, 그래서 다음 줄 코드가 갑자기 에러가 나거나 결과가 달라질 수 있다는 점이다.

## 단일 컬럼 선택

만약 menu 한 개 열만 보고 싶을 때:
- df["menu"] → 보통 Series
    - 특징: “한 줄짜리 컬럼”처럼 보이고, name이 menu로 붙는다.

허나
- Series는 DataFrame처럼 columns가 없다.
- 그래서 "다음 단계에서 DataFrame이라고 생각하고" 코드를 쓰면 오류가 발생한다


예시로 자주 나는 실수:
```python
- df["menu"].columns    # Series에는 columns가 없어서 오류
- df["menu"].merge(...) # merge는 DataFrame 기준이 많아서 흐름이 꼬일 수 있음
```

In [91]:
# ------------------------------------------------------------
# 예시 데이터프레임 만들기 (카페 메뉴/가격)
# ------------------------------------------------------------
df = pd.DataFrame({
    "menu":  ["Latte", "Americano", "Mocha"],
    "price": [5000, 4500, 5500],
    "paid":  [True, True, False]
})

df

,menu,price,paid
0,Latte,5000,True
1,Americano,4500,True
2,Mocha,5500,False


### 단일 컬럼의 선택 : Series가 되는 경우가 많다

In [92]:
# (A) menu 한 개 컬럼만 선택하면 보통 "Series"로 나온다.
#     - Series는 1차원(한 줄짜리 컬럼)처럼 보인다.
menu_series = df["menu"]

# (B) Series의 특징: name 속성이 컬럼명(menu)로 붙는다.
#     - DataFrame에는 columns가 있고,
#     - Series에는 columns가 없다.
menu_series

0        Latte
1    Americano
2        Mocha
Name: menu, dtype: object

In [93]:
type(menu_series)

pandas.core.series.Series

In [94]:
menu_series.name

'menu'

### 무엇이 문제 인가?
Series는 DataFrame처럼 columns가 없기 때문에 "DataFrame이라고 착각하고" 다음 코드를 쓰면 에러가 발생한다.

In [95]:
# 자주 하는 실수 1) Series에 .columns를 접근하려고 함 -> 오류 발생
df["menu"].columns

AttributeError: 'Series' object has no attribute 'columns'

In [96]:
# ------------------------------------------------------------
# 자주 하는 실수 2) merge를 Series에 바로 하려고 함
# - merge는 보통 DataFrame 기준으로 쓰는 경우가 많기 때문에
#   Series로 작업하면 흐름이 꼬이거나 바로 사용이 어렵다.
# - 아래는 "실수 예시"로 일부러 보여주는 코드
# ------------------------------------------------------------

other = pd.DataFrame({
    "menu": ["Latte", "Mocha"],
    "category": ["Milk", "Chocolate"]
})

# Series에는 merge 메서드가 없기 때문에 보통 이런 식으 사용 불가(오류/불편).
df["menu"].merge(other, on="menu")

AttributeError: 'Series' object has no attribute 'merge'

#### (참고) 실무에서 안전한 방식:
- merge가 필요하면, 단일 컬럼이라도 DataFrame으로 유지하는 게 좋다.
- 이중 대괄호를 쓰면 단일 컬럼도 DataFrame으로 유지된다.

In [97]:
menu_df = df[["menu"]]   # <- DataFrame 유지!
menu_df.merge(other, on="menu")

,menu,category
0,Latte,Milk
1,Mocha,Chocolate


## 복수 컬럼 선택 → DataFrame으로 남는 경우가 많다

만약 여러 열을 같이 보고 싶을 때:

- df[["date","menu"]] → DataFrame
- 특징: 표 형태가 그대로 유지되고, columns가 존재한다.

### 실무에서 이게 더 안전한 이유

- 이후 단계(정렬, 저장, merge, 그룹화, 리포트 표 만들기)에서 "표 형태"가 유지돼서 흐름이 안정적

In [ ]:
# ------------------------------------------------------------
# 예시 데이터프레임 만들기 (카페 매출)
# - date, menu 등 여러 컬럼이 있는 "표" 형태
# ------------------------------------------------------------
df = pd.DataFrame({
    "date":  ["2026-01-01", "2026-01-02", "2026-01-03", "2026-01-04"],
    "store": ["A", "A", "B", "A"],
    "menu":  ["Latte", "Americano", "Mocha", "Latte"],
    "price": [5000, 4500, 5500, 5000],
    "qty":   [1, 2, 1, 3]
})

df 

,date,store,menu,price,qty
0,2026-01-01,A,Latte,5000,1
1,2026-01-02,A,Americano,4500,2
2,2026-01-03,B,Mocha,5500,1
3,2026-01-04,A,Latte,5000,3


In [ ]:
# (A) 여러 컬럼을 함께 선택하면 "DataFrame" 형태가 유지된다.
#     - df[["date","menu"]] 처럼 컬럼명을 리스트로 감싸서 선택
#     - 결과는 표(2차원) 그대로이고, columns 속성이 존재한다.
df_small = df[["date", "menu"]]

# DataFrame인지 확인 + columns 확인
df_small, type(df_small), df_small.columns

(         date       menu
 0  2026-01-01      Latte
 1  2026-01-02  Americano
 2  2026-01-03      Mocha
 3  2026-01-04      Latte,
 pandas.core.frame.DataFrame,
 Index(['date', 'menu'], dtype='object'))

#### 실무에서 이게 더 안전한 이유:
"표 형태(DataFrame)"가 유지되면 이후 작업 흐름이 안정적
- 정렬(sort_values)
- 저장(to_csv)
- 결합(merge)
- 그룹화(groupby) 결과 정리
- 리포트용 표 만들기


In [98]:
# 1) 정렬: date 기준으로 정렬 (표 유지라 바로 가능)
sorted_df = df_small.sort_values(by="date")

# 2) 저장: index=False로 깔끔하게 저장 (파일 생성)
# sorted_df.to_csv("report_table.csv", index=False, encoding="utf-8-sig")

# 3) merge: menu 정보를 가진 다른 표와 결합할 때도 DataFrame이 편함
menu_info = pd.DataFrame({
    "menu": ["Latte", "Americano", "Mocha"],
    "category": ["Milk", "Coffee", "Chocolate"]
})

merged_df = df_small.merge(menu_info, on="menu", how="left")

sorted_df, merged_df

(         date       menu
 0  2026-01-01      Latte
 1  2026-01-02  Americano
 2  2026-01-03      Mocha
 3  2026-01-04      Latte,
          date       menu   category
 0  2026-01-01      Latte       Milk
 1  2026-01-02  Americano     Coffee
 2  2026-01-03      Mocha  Chocolate
 3  2026-01-04      Latte       Milk)

## “다음 코드가 달라지는” 대표 사례

### (1) 결과 모양이 달라져서 join/merge/저장이 꼬임
- Series는 저장하면 “열 이름이 애매”해질 수 있고
- DataFrame은 열 이름이 유지되어 파일/리포트에 안정적입니다.

### (2) 그룹화/집계 결과를 이어갈 때
- 단일 열로 뽑아둔 Series는 다음에 컬럼을 추가하거나 결합할 때 불편할 수 있어요.
- DataFrame은 ["colA","colB"] 형태로 확장하기가 쉽습니다.

In [108]:
# ------------------------------------------------------------
# 예시 데이터 (카페 매출)
# ------------------------------------------------------------
df = pd.DataFrame({
    "date":  ["2026-01-01","2026-01-01","2026-01-02","2026-01-02","2026-01-03"],
    "store": ["A","B","A","B","A"],
    "menu":  ["Latte","Latte","Americano","Mocha","Latte"],
    "price": [5000, 5000, 4500, 5500, 5000],
    "qty":   [1, 2, 1, 1, 3]
})

df

,date,store,menu,price,qty
0,2026-01-01,A,Latte,5000,1
1,2026-01-01,B,Latte,5000,2
2,2026-01-02,A,Americano,4500,1
3,2026-01-02,B,Mocha,5500,1
4,2026-01-03,A,Latte,5000,3


#### 대표 사례

##### (1) 결과 모양이 달라져서 join/merge/저장이 꼬이는 경우

In [ ]:
#  단일 컬럼 선택(Series가 되는 경우가 많음)
menu_series = df["menu"]          # Series
#  단일 컬럼이라도 DataFrame으로 유지(이중 대괄호)
menu_df = df[["menu"]]            # DataFrame

- 저장 관점
    - Series는 저장하면 컬럼/헤더가 기대와 다르게 보이거나(특히 name이 없으면) 애매해질 수 있음
    - DataFrame은 columns가 명확히 유지되어 리포트/공유에 안정적

아래는 실제 파일 저장 예시(원하면 주석 해제해서 실행):

In [ ]:
# menu_series.to_csv("menu_series.csv", index=False)  # 결과가 "한 컬럼 파일"처럼 되지만 헤더가 애매할 수 있음
# menu_df.to_csv("menu_df.csv", index=False)          # columns가 유지되어 표 형태가 명확

- merge 관점
    - DataFrame은 merge를 자연스럽게 이어가기 쉬움
    - Series는 merge 메서드가 없어서, 바로 merge하려고 하면 흐름이 끊김(실수 유발)

In [99]:
menu_info = pd.DataFrame({
    "menu": ["Latte", "Americano", "Mocha"],
    "category": ["Milk", "Coffee", "Chocolate"]
})

merged_ok = menu_df.merge(menu_info, on="menu", how="left")  #  자연스러움

# (참고) 아래는 "실수 예시" (주석 해제하면 오류/불편을 체험할 수 있음)
# merged_bad = menu_series.merge(menu_info, on="menu", how="left")  # ❌ Series에는 merge가 없어 보통 오류

type(menu_series), type(menu_df), merged_ok.head()

(pandas.core.series.Series,
 pandas.core.frame.DataFrame,
         menu   category
 0      Latte       Milk
 1  Americano     Coffee
 2      Mocha  Chocolate)

##### (2) 그룹화/집계 결과를 이어갈 때 타입(Series/DF) 때문에 흐름이 달라지는 경우

In [ ]:
# menu별 판매수량 합계 (단일 집계 -> 결과가 Series로 나오기 쉬움)
qty_sum_series = df.groupby("menu")["qty"].sum()  # Series (index=menu, values=합계)

#  여러 집계를 한 번에 하면 DataFrame으로 나오기 쉬움
qty_price_df = df.groupby("menu").agg(
    total_qty=("qty", "sum"),
    avg_price=("price", "mean")
)  # DataFrame

- 왜 Series가 불편할 수 있나?
    - Series는 "열" 개념이 아니라서, 이후에 다른 컬럼을 붙이거나 결합할 때 보통 DataFrame으로 바꿔주는 과정(reset_index / to_frame)이 필요해질 때가 많음.
        - 예: Series를 리포트 표로 만들려고 "menu"를 컬럼으로 만들고 싶으면:
        ```python
        qty_sum_df = qty_sum_series.reset_index(name="total_qty")  # Series -> DataFrame으로 변환(자주 하는 작업)
        ```
- [DataFrame이 편한 이유]
    - 이미 표 형태라서 컬럼 추가/확장이 쉽고, merge/join도 자연스럽게 이어짐
    ```python
    qty_price_df["revenue_est"] = qty_price_df["total_qty"] * qty_price_df["avg_price"]  # 컬럼 추가가 깔끔|
    qty_sum_series, qty_sum_df, qty_price_df
    ```

## 초보자에게 추천하는 실무 습관 2가지

습관 A) “표 형태를 유지하고 싶으면” 항상 이중 대괄호

- 단일 컬럼이어도 DataFrame으로 유지하고 싶으면:
    - df[["menu"]] (이중 대괄호)
        - → 이러면 DataFrame 유지

습관 B) “내가 지금 가진 게 Series인지 DataFrame인지” 한 번만 확인

- type(변수) 또는 .shape로 감 잡기
    - Series: 보통 (행수,)
    - DataFrame: 보통 (행수, 열수)


한 줄 결론

단일 컬럼 선택은 Series가 되어 다음 코드가 달라질 수 있고,

복수 컬럼 선택은 DataFrame으로 남아 흐름이 안정적입니다.

그래서 실무에서는 “표 형태 유지가 필요하면 이중 대괄호”를 습관처럼 씁니다.

### 초보자에게 추천하는 실무 습관 2가지

- A) 표 형태(DataFrame)를 유지하고 싶으면 "이중 대괄호"를 쓰기
- B) 지금 내가 가진 게 Series인지 DataFrame인지 type / shape로 한 번만 확인하기


In [104]:
# 예시 데이터 (카페 매출)
df = pd.DataFrame({
    "menu":  ["Latte", "Americano", "Mocha"],
    "price": [5000, 4500, 5500],
    "paid":  [True, True, False]
})

df

,menu,price,paid
0,Latte,5000,True
1,Americano,4500,True
2,Mocha,5500,False


### A) 표 형태(DataFrame)를 유지하고 싶으면 "이중 대괄호"를 쓰기
- df["menu"]  : 보통 Series (1차원)
- df[["menu"]]: DataFrame (2차원, 표 형태 유지)


In [ ]:
menu_series = df["menu"]     # Series가 되는 경우가 많음
menu_df     = df[["menu"]]   # 이중 대괄호 -> DataFrame 유지

# ------------------------------------------------------------
# 습관 B) 내가 가진 게 Series인지 DataFrame인지 확인하는 방법
# - type(변수): 타입 확인
# - .shape    : 모양 확인
#   * Series   -> (행수,)     # 열 개념이 없어서 1개 값만 나옴
#   * DataFrame-> (행수, 열수) # 표라서 2개 값이 나옴
# ------------------------------------------------------------
check = {
    "menu_series_type": type(menu_series),
    "menu_series_shape": menu_series.shape,   # 예: (3,)
    "menu_df_type": type(menu_df),
    "menu_df_shape": menu_df.shape            # 예: (3, 1)
}

# 노트북에서 print 없이 확인 (마지막 줄 자동 출력)
df, menu_series, menu_df, check


### B) 지금 내가 가진 게 Series인지 DataFrame인지 type / shape로 한 번만 확인하기
- type(변수): 타입 확인
- .shape    : 모양 확인
  - Series   -> (행수,)       # 열 개념이 없어서 1개 값만 나옴
  - DataFrame-> (행수, 열수)  # 표라서 2개 값이 나옴



In [102]:
check = {
    "menu_series_type": type(menu_series),
    "menu_series_shape": menu_series.shape,   # 예: (3,)
    "menu_df_type": type(menu_df),
    "menu_df_shape": menu_df.shape            # 예: (3, 1)
}

# 노트북에서 print 없이 확인 (마지막 줄 자동 출력)
df, menu_series, menu_df, check

(        menu  price   paid
 0      Latte   5000   True
 1  Americano   4500   True
 2      Mocha   5500  False,
 0        Latte
 1    Americano
 2        Mocha
 Name: menu, dtype: object,
         menu
 0      Latte
 1  Americano
 2      Mocha,
 {'menu_series_type': pandas.core.series.Series,
  'menu_series_shape': (3,),
  'menu_df_type': pandas.core.frame.DataFrame,
  'menu_df_shape': (3, 1)})

# 4) 조건 필터링(불리언 인덱싱)의 본질


## 1) 핵심 아이디어: "True/False" 필터를 먼저 만든다
조건 필터링은 사실 2단계로 생각하면 가장 쉽다.
- 1단계. 조건을 만족하면 True, 아니면 False 인 "필터"를 만든다.
- 2단계. 그 필터로 데이터프레임에서 True인 행만 골라낸다

예를 들어 “결제 완료(paid)가 True인 주문만 보고 싶다”면
- paid == True가 맞는 행은 True, 아닌 행은 False가 된다.

이 True/False 배열이 바로 필터이다.

### 결론(한 줄)
- 조건 필터링은 True/False 필터를 만든 다음 적용하는 방식
- 다중 조건일 경우 괄호 필수 + &|~ 사용

- 실무에서는 df.loc[조건, 컬럼] 형태가 가장 안전한 표준


In [ ]:
#  ------------------------------------------------------------
# 실무에서 가장 안전한 기본 형태: df.loc[조건, 컬럼]
#
# 왜 안전한가?
# - 조건(행 선택)과 컬럼(열 선택)의 역할이 한 줄에 "명확히" 나눠진다.
#   * 조건  : 어떤 행을 고를지 (필터 기준)
#   * 컬럼  : 어떤 열만 보여줄지 (리포트/분석에 필요한 정보만)
#
# 예: "결제 완료인 행만" + "date, menu, price만 보여줘"
# ------------------------------------------------------------

df = pd.DataFrame({
    "date":  ["2026-01-01","2026-01-01","2026-01-02","2026-01-03"],
    "menu":  ["Latte","Americano","Mocha","Latte"],
    "price": [5000, 4500, 5500, 5000],
    "qty":   [1, 2, 1, 3],
    "paid":  [True, False, True, True]
})

# 1) 조건(행을 고르는 기준) 만들기: 결제 완료(paid == True)만
condition = (df["paid"] == True)

# 2) 컬럼(보여줄 열) 선택: date, menu, price만
cols = ["date", "menu", "price"]

# 3) 안전한 기본 형태: df.loc[조건, 컬럼]
result = df.loc[condition, cols]

# 노트북에서는 마지막 줄에 두면 print 없이 자동 출력
df, result

## 2) 왜 굳이 True/False 필터를 만들까?
"바로 뽑으면 되는거 아닌가?"라 생각할수도 있다.

허나 필터란 기능은 생각한것보다 유능한 점이 많이 있다.
- 조건이 맞는 행이 몇 개나 되는지 바로 확인 가능
- 여러 조건을 조합할 때*디버깅이 쉬움
- 필터를 저장해두면 같은 조건을 재사용하기 좋음

즉, “필터 만들기 → 적용” 구조는 실수를 줄이는 안전장치라 할수 있다.

In [ ]:
# ------------------------------------------------------------
# 왜 굳이 True/False 필터(마스크)를 "따로" 만들까?
# 1) 조건에 맞는 행이 "몇 개인지" 바로 확인 가능
# 2) 여러 조건을 조합할 때 디버깅(어디서 걸렸는지 확인)이 쉬움
# 3) 필터를 변수로 저장해두면 같은 조건을 재사용하기 좋음
# ------------------------------------------------------------

df = pd.DataFrame({
    "order_id": [1, 2, 3, 4, 5],
    "store":    ["A","A","B","A","B"],
    "menu":     ["Latte","Americano","Latte","Mocha","Latte"],
    "price":    [5000, 4500, 5000, 5500, 5000],
    "paid":     [True, False, True, True, False]
})

In [ ]:
# ------------------------------------------------------------
# (1) 필터를 분리하면: "조건에 맞는 행이 몇 개인지" 바로 확인 가능
# ------------------------------------------------------------
mask_paid = (df["paid"] == True)      # 결제 완료만 True
count_paid = mask_paid.sum()          # True는 1로 취급되므로 합계 = True 개수

In [ ]:
# ------------------------------------------------------------
# (2) 여러 조건을 조합할 때: 조건을 쪼개서 디버깅이 쉬움
# 예: "결제 완료" AND "A지점" AND "Latte"
# - 각각의 조건이 얼마나 걸러지는지 중간중간 확인 가능
# ------------------------------------------------------------
mask_storeA = (df["store"] == "A")
mask_latte  = (df["menu"] == "Latte")

# 조합 필터 (괄호 필수!)
mask_final = mask_paid & mask_storeA & mask_latte

In [ ]:
# ------------------------------------------------------------
# (3) 필터를 저장해두면: 같은 조건을 재사용하기 좋음
# - 같은 mask_final을 가지고 "필요한 컬럼만" 뽑거나
# - 다른 분석(정렬/집계)로 이어갈 수 있음
# ------------------------------------------------------------
result_all_cols = df[mask_final]                            # 전체 컬럼
result_some_cols = df.loc[mask_final, ["order_id","menu","price"]]  # 필요한 컬럼만

# 노트북에서는 마지막 줄에 두면 print 없이 자동 출력
df, count_paid, mask_final, result_all_cols, result_some_cols

## 3) 다중 조건에서 꼭 지켜야 하는 규칙(가장 많이 틀리는 부분)
조건이 여러개로 늘어나는 경우 당연스럽게도 복잡해진다.

때문에 작성 플롯을 어느정도 정해두고 문제에 임하는게 좋다.

여러 조건을 함께 쓰는 순간부터 실수율이 확 올라갑니다.

1) &(AND), |(OR), ~(NOT) 사용할 때는 괄호 필수
    - AND: (조건1) & (조건2)
    - OR: (조건1) | (조건2)
    - NOT: ~(조건)
    
괄호를 사용하지 않으면 파이썬이 조건의 우선순위를 다르게 해석해 에러가 나거나, 의도와 다른 결과가 나오기 쉽다.

2) and, or가 아니라 &, |를 쓴다
    - and/or는 “파이썬 단일 True/False”에 쓰는 경우가 많다.
    - 판다스 필터(배열)에는 &, | 를 쓰는 게 기본

In [105]:
# ------------------------------------------------------------
# 다중 조건 필터링 규칙(초보자가 가장 많이 틀리는 부분)
#
# 규칙 1) &(AND), |(OR), ~(NOT) 사용할 때는 "괄호 필수"
#   - AND: (조건1) & (조건2)
#   - OR : (조건1) | (조건2)
#   - NOT: ~(조건)
#
# 규칙 2) and/or가 아니라 &/| 를 쓴다
#   - and/or는 "단일 True/False"에 쓰는 경우가 많음
#   - 판다스의 조건 필터(여러 행에 대한 True/False 배열)에는 &/| 를 사용
# ------------------------------------------------------------

df = pd.DataFrame({
    "order_id": [1, 2, 3, 4, 5, 6],
    "store":    ["A","A","B","A","B","A"],
    "menu":     ["Latte","Americano","Latte","Mocha","Latte","Latte"],
    "price":    [5000, 4500, 5000, 5500, 5000, 5000],
    "paid":     [True, False, True, True, False, True]
})

In [ ]:
# ------------------------------------------------------------
# 올바른 예시 1) AND: 결제완료 & A지점 & Latte
# - 괄호 필수!
# ------------------------------------------------------------
mask_and = (df["paid"] == True) & (df["store"] == "A") & (df["menu"] == "Latte")
and_result = df.loc[mask_and, ["order_id", "store", "menu", "price"]]

In [ ]:
# ------------------------------------------------------------
# 올바른 예시 2) OR: Latte 또는 Mocha 메뉴
# - 괄호 필수!
# ------------------------------------------------------------
mask_or = (df["menu"] == "Latte") | (df["menu"] == "Mocha")
or_result = df.loc[mask_or, ["order_id", "menu", "paid"]]


In [ ]:
# ------------------------------------------------------------
# 올바른 예시 3) NOT: 결제 실패(paid가 False)만 보기
# - NOT은 ~(조건) 형태
# ------------------------------------------------------------
mask_not = ~(df["paid"] == True)   # paid가 True가 아닌 것 -> False인 것들
not_result = df.loc[mask_not, ["order_id", "paid", "menu"]]


In [ ]:
# ------------------------------------------------------------
# ❌ 잘못된 예시(설명용): and/or를 쓰면 보통 에러가 나거나 의도대로 안 됩니다.
# - 판다스 조건은 "여러 행의 True/False 배열"인데
#   and/or는 단일 True/False처럼 평가하려고 해서 문제가 생김
#
# 아래 줄은 오류가 나기 때문에 주석으로만 남깁니다.
# wrong = df[(df["paid"] == True) and (df["store"] == "A")]
# ------------------------------------------------------------

# 노트북에서는 마지막 줄에 두면 print 없이 자동 출력
df, and_result, or_result, not_result

## 4) 실무에서 가장 안전한 기본 형태: df.loc[조건, 컬럼]

조건 필터링에서 실무 기본 문장은 이 형태이다
```python
df.loc[조건, 컬럼]
```

이게 안전한 이유는 한 문장에 역할이 분명하게 나뉘기 때문이다.

- 조건 : “행을 고르는 기준”
- 컬럼 : “보여줄 열만 선택”

즉, 행 선택과 열 선택을 동시에, 명확하게 처리된다.

예시를 한번 보면
- "결제 완료인 행만" + "date, menu, price만 보여줘"
- 같은 요구를 가장 깔끔하게 표현하는 방식이 .loc이다.

In [106]:
# ------------------------------------------------------------
# 실무에서 가장 안전한 기본 형태: df.loc[조건, 컬럼]
#
# 왜 안전한가?
# - 조건(행 선택)과 컬럼(열 선택)의 역할이 한 줄에 "명확히" 나눠진다.
#   * 조건  : 어떤 행을 고를지 (필터 기준)
#   * 컬럼  : 어떤 열만 보여줄지 (리포트/분석에 필요한 정보만)
#
# 예: "결제 완료인 행만" + "date, menu, price만 보여줘"
# ------------------------------------------------------------

df = pd.DataFrame({
    "date":  ["2026-01-01","2026-01-01","2026-01-02","2026-01-03"],
    "menu":  ["Latte","Americano","Mocha","Latte"],
    "price": [5000, 4500, 5500, 5000],
    "qty":   [1, 2, 1, 3],
    "paid":  [True, False, True, True]
})

df

,date,menu,price,qty,paid
0,2026-01-01,Latte,5000,1,True
1,2026-01-01,Americano,4500,2,False
2,2026-01-02,Mocha,5500,1,True
3,2026-01-03,Latte,5000,3,True


In [ ]:
# 1) 조건(행을 고르는 기준) 만들기: 결제 완료(paid == True)만
condition = (df["paid"] == True)

# 2) 컬럼(보여줄 열) 선택: date, menu, price만
cols = ["date", "menu", "price"]

# 3) 안전한 기본 형태: df.loc[조건, 컬럼]
result = df.loc[condition, cols]

df, result

(         date       menu  price  qty   paid
 0  2026-01-01      Latte   5000    1   True
 1  2026-01-01  Americano   4500    2  False
 2  2026-01-02      Mocha   5500    1   True
 3  2026-01-03      Latte   5000    3   True,
          date   menu  price
 0  2026-01-01  Latte   5000
 2  2026-01-02  Mocha   5500
 3  2026-01-03  Latte   5000)

## 5) 결론

- 조건 필터링은 True/False 필터를 만든 다음 적용하는 방식
- 다중 조건은 괄호 필수 + &|~ 사용
- 실무에서는 df.loc[조건, 컬럼] 형태가 가장 안전한 표준

# 5) 정렬(sort_values, sort_index)이 필요한 이유

정렬은 한마디로 "분석 결과를 사람이 읽기 좋은 순서로 재배치하는 작업" 이다.

데이터를 가공(필터링/집계)해도, 정렬이 없으면 표가 뒤죽박죽이라 핵심이 안 보이는 경우가 있다.

## 1) 보고서/리포트는 "정렬된 표"가 기본

실무에서 보고서를 만들 때는 거의 항상 이런 순서를 요구한다.

- 최신순: "최근 데이터부터 보여줘" (날짜 기준 내림차순)
- 매출 큰 순: "어떤 메뉴/지점이 매출을 많이 만들었지?" (매출 기준 내림차순)
- 수량 많은 순: "가장 많이 팔린 메뉴 TOP N" (판매수량 기준 내림차순)


정렬은 정리의 의미를 가지며 그 목적은 "의사결정에 필요한 우선순위를 보여주기" 위함이라 할수있다.

In [109]:
# 샘플: 카페 주문 데이터
df = pd.DataFrame({
    "date": ["2026-01-03", "2026-01-01", "2026-01-02", "2026-01-03", "2026-01-01"],
    "menu": ["Latte", "Americano", "Mocha", "Americano", "Latte"],
    "qty":  [2, 1, 3, 1, 1],
    "price":[5000, 4500, 5500, 4500, 5000]
})

# 매출 컬럼(수량 * 가격) 추가
df["revenue"] = df["qty"] * df["price"]

# 1) 최신순: 날짜 기준 내림차순
latest_first = df.sort_values(by="date", ascending=False)

# 2) 매출 큰 순: revenue 기준 내림차순
revenue_first = df.sort_values(by="revenue", ascending=False)

# 3) 수량 많은 순: qty 기준 내림차순 (TOP 3만)
top_qty = df.sort_values(by="qty", ascending=False).head(3)

df, latest_first, revenue_first, top_qty

(         date       menu  qty  price  revenue
 0  2026-01-03      Latte    2   5000    10000
 1  2026-01-01  Americano    1   4500     4500
 2  2026-01-02      Mocha    3   5500    16500
 3  2026-01-03  Americano    1   4500     4500
 4  2026-01-01      Latte    1   5000     5000,
          date       menu  qty  price  revenue
 0  2026-01-03      Latte    2   5000    10000
 3  2026-01-03  Americano    1   4500     4500
 2  2026-01-02      Mocha    3   5500    16500
 1  2026-01-01  Americano    1   4500     4500
 4  2026-01-01      Latte    1   5000     5000,
          date       menu  qty  price  revenue
 2  2026-01-02      Mocha    3   5500    16500
 0  2026-01-03      Latte    2   5000    10000
 4  2026-01-01      Latte    1   5000     5000
 1  2026-01-01  Americano    1   4500     4500
 3  2026-01-03  Americano    1   4500     4500,
          date       menu  qty  price  revenue
 2  2026-01-02      Mocha    3   5500    16500
 0  2026-01-03      Latte    2   5000    10000
 1  2026-0

## 2) 정렬을 하면 인사이트가 바로 보인다

예를 들어 메뉴별 매출 집계표가 있을 때 정렬을 안 하면:

- 라떼가 1등인지 5등인지 눈으로 찾기 어려움
- 중요 메뉴/문제 메뉴가 어디인지 한 번에 안 보임

반대로 매출 내림차순 정렬을 하면:

- 상위 메뉴 TOP 5가 즉시 보임
- "우리가 밀어야 할 메뉴"가 명확해짐
- 하위 메뉴도 바로 보여서 "개선 대상"을 찾기 쉬워짐

In [110]:
# 샘플: 카페 주문 데이터
df = pd.DataFrame({
    "menu": ["Latte","Americano","Mocha","Latte","Mocha","Americano","Tea","Tea","Latte"],
    "qty":  [2, 1, 1, 3, 2, 4, 5, 1, 1],
    "price":[5000,4500,5500,5000,5500,4500,4000,4000,5000]
})

# 매출(=수량*가격) 컬럼 추가
df["revenue"] = df["qty"] * df["price"]

# 메뉴별 매출 집계표 만들기
menu_sales = df.groupby("menu", as_index=False)["revenue"].sum()

# 1) 정렬 안 한 집계표: 순서가 애매해서 TOP 메뉴가 바로 안 보일 수 있음
menu_sales_unsorted = menu_sales

# 2) 매출 내림차순 정렬: TOP 메뉴가 즉시 보임 (TOP 5)
menu_sales_sorted = menu_sales.sort_values(by="revenue", ascending=False)
top5 = menu_sales_sorted.head(5)

# 3) 하위 메뉴(개선 대상)도 바로 보임 (BOTTOM 3)
bottom3 = menu_sales_sorted.tail(3)

menu_sales_unsorted, top5, bottom3

(        menu  revenue
 0  Americano    22500
 1      Latte    30000
 2      Mocha    16500
 3        Tea    24000,
         menu  revenue
 1      Latte    30000
 3        Tea    24000
 0  Americano    22500
 2      Mocha    16500,
         menu  revenue
 3        Tea    24000
 0  Americano    22500
 2      Mocha    16500)

## 3) 정렬은 “커뮤니케이션 비용”을 줄인다.

분석 결과는 보통 다른 사람(팀장/클라이언트/동료) 에게 보여야 한다.

이때 정렬이 되어 있으면 보는 사람이:

- “뭘 먼저 봐야 하는지” 바로 이해
- 질문이 줄어듬
- 보고서 설득력이 올라감

정렬이 안 되어 있으면 반대로

"이게 중요한 결과 맞나요?" 나 "제가 이렇게 이해 했는데 맞나요?" 결과에 대한 질문이 늘어난다.

In [111]:
# 예시: "팀장에게 메뉴별 매출 리포트"를 보여주는 상황
# - 정렬이 안 된 표(unsorted)는 핵심이 바로 안 보여서 질문이 늘 수 있다.
# - 정렬된 표(sorted)는 TOP 메뉴가 바로 보여서 커뮤니케이션이 쉬워진다.

df = pd.DataFrame({
    "menu": ["Latte", "Americano", "Mocha", "Tea", "Cake"],
    "revenue": [32000, 15000, 27000, 9000, 12000]  # 메뉴별 총매출(이미 집계된 값이라고 가정)
})

# 1) 정렬 안 한 리포트: 보는 사람이 "뭐가 1등이지?"를 눈으로 찾아야 함
report_unsorted = df

# 2) 매출 내림차순 정렬된 리포트: TOP 메뉴가 위에 고정되어 바로 이해됨
report_sorted = df.sort_values(by="revenue", ascending=False)

# 3) 보통 보고서에는 TOP N을 같이 제시하면 설득력이 더 좋아짐
top3 = report_sorted.head(3)

report_unsorted, report_sorted, top3

(        menu  revenue
 0      Latte    32000
 1  Americano    15000
 2      Mocha    27000
 3        Tea     9000
 4       Cake    12000,
         menu  revenue
 0      Latte    32000
 2      Mocha    27000
 1  Americano    15000
 4       Cake    12000
 3        Tea     9000,
         menu  revenue
 0      Latte    32000
 2      Mocha    27000
 1  Americano    15000)

## 4) sort_values vs sort_index 차이(감 잡기)

- sort_values: "값" 기준 정렬
- 예: 매출, 수량, 평점, 가격 순서로 정렬할 때
- sort_index: "인덱스(행 이름표):" 기준 정렬
- 예: 날짜를 인덱스로 두었을 때 날짜순으로 정렬하고 싶을 때
- 예: 그룹화 결과가 인덱스가 되어 있을 때 정리할 때

리포트/랭킹표는 대부분 sort_values가 더 자주 쓰이고, 

시간축 데이터/인덱스 기반 표 정리는 sort_index가 자주 쓰인다.

한 줄 결론

정렬은 분석 결과를 “읽히는 결과”로 바꾸는 마지막 기본 단계, 리포트에서 요구하는 최신순/Top N/우선순위 표는 대부분 정렬로 완성된다.

In [ ]:
# ------------------------------------------------------------
# sort_values vs sort_index 차이 한 번에 감 잡기
# - sort_values: "값" 기준 정렬 (리포트/랭킹표에서 가장 자주)
# - sort_index : "인덱스(행 이름표)" 기준 정렬 (시간축/인덱스 기반 표 정리)
# ------------------------------------------------------------

# 1) sort_values 예시: 메뉴별 매출(값)로 정렬
sales = pd.DataFrame({
    "menu": ["Latte", "Americano", "Mocha", "Tea"],
    "revenue": [32000, 15000, 27000, 9000]
})

# 값(revenue) 기준 내림차순 정렬 -> "매출 TOP" 랭킹표 만들 때
sales_sorted_by_value = sales.sort_values(by="revenue", ascending=False)


# 2) sort_index 예시: 날짜를 인덱스로 둔 뒤 인덱스(날짜 라벨)로 정렬
daily = pd.DataFrame({
    "date": ["2026-01-03", "2026-01-01", "2026-01-02"],
    "revenue": [12000, 8000, 15000]
}).set_index("date")   # date가 인덱스(행 이름표)가 됨

# 인덱스(날짜) 기준 오름차순 정렬 -> 시간 흐름대로 정리할 때
daily_sorted_by_index = daily.sort_index(ascending=True)


# 3) sort_index 예시(그룹화 결과 정리): groupby 결과는 인덱스가 menu가 되는 경우가 많음
orders = pd.DataFrame({
    "menu": ["Latte","Americano","Latte","Mocha","Mocha","Tea"],
    "qty":  [2, 1, 3, 1, 2, 4]
})

qty_sum = orders.groupby("menu")["qty"].sum()  # 결과: 인덱스가 menu인 Series
qty_sum_sorted_by_index = qty_sum.sort_index() # 인덱스(메뉴 이름) 알파벳/가나다 순 정리


sales, sales_sorted_by_value, daily, daily_sorted_by_index, qty_sum, qty_sum_sorted_by_index

# 6) 클리닝(정제)의 4대 문제

컬럼명/구조 문제
- 컬럼명/구조 문제: rename, drop
- 문자열 문제: 공백, 대소문자, 불필요 문자(“원”, “,”)
- 결측치 문제: NaN(비어 있음)
- 중복 문제: 같은 행이 여러 번 있음

결측치 전략 2가지(초보자 기준)

- dropna: "신뢰할 수 없는 행은 버린다"(단순)
- fillna: "합리적 값으로 채운다"(실무형)
- 평균/중앙값/그룹별 평균 등

중복 처리에서 중요한 옵션

- subset: 무엇을 기준으로 중복인지 판단할지
- keep: 첫 번째를 남길지, 마지막을 남길지

In [112]:
# ------------------------------------------------------------
# 초보자 실습용 "클리닝(정제) 4대 문제" 미니 샘플
# - 컬럼명/구조 문제: rename, drop
# - 문자열 문제: 공백, 대소문자, "원", "," 같은 불필요 문자 제거
# - 결측치 문제: NaN 처리(dropna / fillna)
# - 중복 문제: duplicated / drop_duplicates (subset, keep)
# ------------------------------------------------------------

# 일부러 문제를 섞어 만든 원본 데이터
raw = pd.DataFrame({
    " Date ": ["2026-01-01", "2026-01-01", "2026-01-02", None,        "2026-01-02"],
    " Menu ": [" Latte ",   " latte",      "Americano ", "Mocha",     "Americano "],
    " Price ": ["5,000원",   "5,000원",     None,         "5,500원",   None],
    " Qty ":   [1,           1,            2,            None,        2],
    " memo ":  ["test",      "test",       "dup",        "x",         "dup"]  # 분석에 필요 없는 컬럼이라고 가정
})

raw

,Date,Menu,Price,Qty,memo
0,2026-01-01,Latte,"5,000원",1.0,test
1,2026-01-01,latte,"5,000원",1.0,test
2,2026-01-02,Americano,None,2.0,dup
3,None,Mocha,"5,500원",NaN,x
4,2026-01-02,Americano,None,2.0,dup


### 컬럼명/구조 문제: rename, drop
 - 컬럼명 공백 제거 + 소문자 통일
 - 필요 없는 컬럼(memo) 제거

In [114]:
df = raw.copy()

df.columns = df.columns.str.strip().str.lower()   # " Date " -> "date"
df = df.rename(columns={"menu": "menu_name"})     # 예: menu -> menu_name
df = df.drop(columns=["memo"])                   # 필요 없는 컬럼 제거

df

,date,menu_name,price,qty
0,2026-01-01,Latte,"5,000원",1.0
1,2026-01-01,latte,"5,000원",1.0
2,2026-01-02,Americano,None,2.0
3,None,Mocha,"5,500원",NaN
4,2026-01-02,Americano,None,2.0


### 문자열 문제: 공백/대소문자/불필요 문자 제거
 - menu_name: 앞뒤 공백 제거 + 소문자 통일
 - price: "원", "," 제거 후 숫자로 변환

In [115]:
# menu_name 정리
df["menu_name"] = df["menu_name"].str.strip().str.lower()

# price 정리: "5,000원" -> "5000" -> 5000 (숫자)
df["price"] = (
    df["price"]
    .astype("string")                    # None도 다루기 쉽게
    .str.replace(",", "", regex=False)
    .str.replace("원", "", regex=False)
)

# 숫자로 변환 (변환 불가한 값은 NaN)
df["price"] = pd.to_numeric(df["price"], errors="coerce")

df

,date,menu_name,price,qty
0,2026-01-01,latte,5000,1.0
1,2026-01-01,latte,5000,1.0
2,2026-01-02,americano,<NA>,2.0
3,None,mocha,5500,NaN
4,2026-01-02,americano,<NA>,2.0


### 결측치(NaN) 문제: dropna vs fillna
- dropna: 비어있는 행은 버린다(단순)
- fillna: 합리적인 값으로 채운다(실무형)

In [116]:
# 1) 단순 전략: price 또는 qty가 비어있으면 행을 버리기
df_drop = df.dropna(subset=["price", "qty"])

# 2) 실무형 전략: 결측치 채우기
# - qty: 비어있으면 1로 채운다고 가정(주문 수량 기본값)
# - price: 메뉴별 평균 가격으로 채우기 (groupby 사용)
df_fill = df.copy()

df_fill["qty"] = df_fill["qty"].fillna(1)

# 메뉴별 평균 가격(메뉴별로 price 결측을 채우기 위한 기준)
menu_mean_price = df_fill.groupby("menu_name")["price"].transform("mean")
df_fill["price"] = df_fill["price"].fillna(menu_mean_price)

df_drop, df_fill


(         date menu_name  price  qty
 0  2026-01-01     latte   5000  1.0
 1  2026-01-01     latte   5000  1.0,
          date  menu_name  price  qty
 0  2026-01-01      latte   5000  1.0
 1  2026-01-01      latte   5000  1.0
 2  2026-01-02  americano   <NA>  2.0
 3        None      mocha   5500  1.0
 4  2026-01-02  americano   <NA>  2.0)

### 중복 문제: duplicated / drop_duplicates
- subset: 무엇을 기준으로 중복인지 판단할지
- keep: "first" (첫 행 유지) or "last" (마지막 행 유지)

In [117]:
# 중복 기준 예시: date + menu_name + qty + price 가 같으면 같은 주문으로 보자
dup_mask = df_fill.duplicated(subset=["date", "menu_name", "qty", "price"], keep="first")

# 중복 제거(첫 번째 유지)
dedup_first = df_fill.drop_duplicates(subset=["date", "menu_name", "qty", "price"], keep="first")

# 중복 제거(마지막 유지)
dedup_last = df_fill.drop_duplicates(subset=["date", "menu_name", "qty", "price"], keep="last")

dup_mask, dedup_first, dedup_last

(0    False
 1     True
 2    False
 3    False
 4     True
 dtype: bool,
          date  menu_name  price  qty
 0  2026-01-01      latte   5000  1.0
 2  2026-01-02  americano   <NA>  2.0
 3        None      mocha   5500  1.0,
          date  menu_name  price  qty
 1  2026-01-01      latte   5000  1.0
 3        None      mocha   5500  1.0
 4  2026-01-02  americano   <NA>  2.0)